Preparación datos

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [7]:
#fests_22 = pd.read_csv("./data/2022_festivals-assistents-order-name.csv")
#fuente_1 = "https://opendata-ajuntament.barcelona.cat/data/es/dataset/dades-festivals/resource/c573360d-ed5d-4609-aaa5-55bb9e9175ca#additional-info"
fests_21 = pd.read_csv("./data/2021_festivals-assistents-order-name.csv")
fuente_2 = "https://opendata-ajuntament.barcelona.cat/data/es/dataset/dades-festivals/resource/57ed9976-8291-418d-a0fc-ecf572d8cb74?inner_span=True"
fests_20= pd.read_csv("./data/2020_festivals-assistents-order-name.csv")
fuente_3 = "https://opendata-ajuntament.barcelona.cat/data/es/dataset/dades-festivals/resource/1214655c-38e5-4434-a35c-ed3a6468cd23?inner_span=True"
fests_19 = pd.read_csv("./data/2019_festivals-assistents-order-name.csv")
fuente_4 = "https://opendata-ajuntament.barcelona.cat/data/es/dataset/dades-festivals/resource/077d738b-997f-46b9-ab96-500ab22f32b6?inner_span=True"
fests_18 = pd.read_csv("./data/2018_festivals-assistents-order-name.csv")
fuente_5 = "https://opendata-ajuntament.barcelona.cat/data/es/dataset/dades-festivals/resource/a5522295-70f1-4d0b-8cb2-8762cbcbf783?inner_span=True"

FileNotFoundError: [Errno 2] No such file or directory: './data/2021_festivals-assistents-order-name.csv'

In [ ]:
def eliminar_festis(df):
    festis_out = [
        '48H Open House BCN', 'ARQUINSET', 'Barcelona Pensa', 'Biennal de pensament Ciutat Oberta',
        'DAU Barcelona', 'Festival IF Barcelona', 'Festival Sant Jordi de Nadal', 'Festival TRO',
        'Fixing the Future', 'FLIB. Festival de Literatura Italiana a Barcelona',
        'Flic. Festival de literatures i arts infantil i juvenil', 'Il·luLola. Festival d¿il·lustració infantil i juvenil',
        "Inrisus, Festival d'Humor de Barcelona", 'KMAmèrica. Festival de Literatura Llatinoamericana de Barcelona',
        "La setmana de l'àlbum", 'La Setmana de la Ciència', 'Liternatura. Festival de Literatura de Natura',
        'Més que Muses. Cicle de Dones Creadores', "Model. Festival d'Arquitectures de Barcelona",
        'Poblenou Open Day', 'Poblenou Open Night', 'Qlit. Festival de Literatura Queer', 'Séfer Festival del Libro Judío', 
        "Toon a Ville", "Transeuropa Festival", "Festival Internacional d'Arquitectura eme3", "Sant Jordi al Raval", "Primavera Vaca", 
        "Músics al Metro de Barcelona", "Como Pedro por mi casa", "Festival de la Ciència", "Som Cultura", "Festival Vèrtex BCN",
        "Palo Market Fest", "Llindars, Festival d'instal·lació i intervenci...", "Moviestar Liternatura", "Biennal de pensament Ciutat Oberta",
        "Biennal Ciutat i Ciència", "De Flor en Flor", "Festival del Llibre d'Artista i la Petita Edició", "GRAF. Còmic d'autor i edició independent",
        "Marrameu - Festival Jove de Nit d'Ànimes de la Guineueta", "MemeFest", "Novembre Vaca", "Smart City Week", 
        "Biennal de pensament Ciutat Oberta", "Barcelona Novel·la Històrica"
    ]
    
    filtro = ~df['NomDelFestival'].isin(festis_out)
    return df[filtro]

In [ ]:
def eliminar_columnas(df):
    columnas_a_eliminar = ["DataFi", "Any", "Organitzador", "Notes_Dades", "Tipus", "Espai"]
    df.drop(columns=columnas_a_eliminar, inplace=True)
    return df

In [ ]:
def clasificar_titularidad(df, columna_entrada='Titularitat', columna_salida='Titularidad'):
    Titul = []
    for tit in df[columna_entrada]:
        if tit == "Ajuntament de Barcelona":
            Titul.append("Pública")
        elif tit == "Titularitat privada":
            Titul.append("Privada")
        elif tit == "Altres administracions públiques":
            Titul.append("Pública")
        elif tit == "Consorci o fundació amb presència administracions públiques":
            Titul.append("Mixta")
        else:
            Titul.append("Otro")
    
    df[columna_salida] = Titul
    df.drop(columns=columna_entrada, inplace=True)

    return df

In [ ]:
def total_asist(df):
    # Convertimos los valores de asistencia a float para asegurar la correcta suma
    df['Assistents_a_Barcelona'] = df['Assistents_a_Barcelona'].fillna(0).astype(float)
    df['Assistents_fora_de_Barcelona'] = df['Assistents_fora_de_Barcelona'].fillna(0).astype(float)

    # Sumamos los dos tipos de asistentes y aseguramos que el total tampoco tenga NaN
    df['total_asist'] = df['Assistents_a_Barcelona'] + df['Assistents_fora_de_Barcelona']
    df['total_asist'] = df['total_asist'].fillna(0).astype(float)  # Convertir a float para asegurar el tipo de dato

    return df

In [ ]:
def cuando_mes_estacion(df):
    df['DataInici'] = pd.to_datetime(df['DataInici'], format='%d/%m/%Y')
    df['Mes'] = df['DataInici'].dt.month_name(locale='es_ES').str.capitalize()

    def que_estacion(date):
        year = date.year
        if (date >= pd.Timestamp(year=year, month=12, day=21)) or (date < pd.Timestamp(year=year, month=3, day=21)):
            return 'Invierno'
        elif (date >= pd.Timestamp(year=year, month=3, day=21)) and (date < pd.Timestamp(year=year, month=6, day=21)):
            return 'Primavera'
        elif (date >= pd.Timestamp(year=year, month=6, day=21)) and (date < pd.Timestamp(year=year, month=9, day=21)):
            return 'Verano'
        else:
            return 'Otoño'
    
    df['Estacion'] = df['DataInici'].apply(que_estacion)
    df.drop('DataInici', axis=1, inplace=True)

    return df

In [ ]:
def procesar_festival(df_original):
    
    df = df_original.copy()
    
    df.drop(df[df['Tipus'] == 'Fires'].index, inplace=True)

    df = clasificar_titularidad(df)

    df = eliminar_columnas(df)

    df = eliminar_festis(df)

    df = cuando_mes_estacion(df)

    df = total_asist(df)

    return df

In [ ]:
def top_10_festivals(df):
    return df.nlargest(10, 'Assistents_a_Barcelona')[['NomDelFestival', 'Assistents_a_Barcelona']]

In [ ]:
def crear_df_final(original_df, festival_list):
    return original_df[original_df['NomDelFestival'].isin(festival_list)]

In [ ]:
def reset_index(df):
    df_reset = df.reset_index(drop=True)
    df_reset.index += 1
    return df_reset

In [ ]:
festivales_bcn_22 = procesar_festival(fests_22)
festivales_bcn_21 = procesar_festival(fests_21)
festivales_bcn_20 = procesar_festival(fests_20)
festivales_bcn_19 = procesar_festival(fests_19)
festivales_bcn_18 = procesar_festival(fests_18)


Seleccion de los más relevantes según asistentes y presentes en la mayoría de los años

In [ ]:
top_festis_22 = top_10_festivals(festivales_bcn_22)
top_festis_21 = top_10_festivals(festivales_bcn_21)
top_festis_20 = top_10_festivals(festivales_bcn_20)
top_festis_19 = top_10_festivals(festivales_bcn_19)
top_festis_18 = top_10_festivals(festivales_bcn_18)

In [ ]:
fests_mas_top = pd.concat([top_festis_22, top_festis_21, top_festis_20, top_festis_19, top_festis_18])
festis_repes = fests_mas_top['NomDelFestival'].value_counts()[lambda x: x >= 3].index.tolist()

In [ ]:
festivales_bcn_22 = reset_index(crear_df_final(festivales_bcn_22, festis_repes))
festivales_bcn_21 = reset_index(crear_df_final(festivales_bcn_21, festis_repes))
festivales_bcn_20 = reset_index(crear_df_final(festivales_bcn_21, festis_repes))
festivales_bcn_19 = reset_index(crear_df_final(festivales_bcn_19, festis_repes))
festivales_bcn_18 = reset_index(crear_df_final(festivales_bcn_18, festis_repes))

Ahora, los DataFrames modificados y preparados para procesar, y una descripción de los mismos:

In [ ]:
**Dataset:**  festivales_bcn_[año] <br>
**Descripción:**  Se trata de datasets obtenidos de los datos abiertos del Ayuntamiento de Barcelona con el registro de festivales culturales en la ciudad <br>
**Periodo:**  Cada dataset corresponde a un año diferente <br>

|Columna/Variable|Descripción|Tipo_de_Variable|Importancia inicial|Nota|
|-|-|-|-|-|
|Col1, sin nombre|Es el índice de cada dataframe, comenzando en 1...|Numérica(int)|--|sólo índice||
|Ambit|Ámbito artístico al que pertenece cada festival|Categórica nominal (Texto(string))|0|Cardinalidad_ 6||
|Edició|Número de ediciones de ese festival|Numérica discreta(float)|2|número de la edición del año correspondiente||
|NomDelFestival|Nombre del festival|Categórica nominal (Texto(string))|3|||
|Assistents_a_Barcelona|Número de asistentes de Barcelona|Numérica continua(float)|0|Dato no siempre disponible||
|Assistents_fora_de_Barcelona|Número de asistentes de fuera de Barcelona|Numérica continua(float)|0|Dato pocas veces disponible||
|Tutularidad|Entidad responsable de la organización del festival|Categórica nominal (Texto(string))|1|Datos a partir de la transformación de la columna Titularitat original que tenía más variables||
|Mes|El mes al que corresponde el dataset|Categórica nominal (Texto(string))|2|Datos a partir de la transformación de la columna DataInici, luego eliminada||
|Estacion|La estación al que corresponde el dataset|Categórica nominal (Texto(string))|1|Datos a partir de la transformación de la columna DataInici, luego eliminada||

In [ ]:
categoricas_festivales = [
    "Ambit",
    "Titularidad",
    "Mes",
    "Estacion"
]


In [ ]:
categoricas_destacadas = [
    'Ambit', 
    'Titularidad', 
    'Mes', 
    'Estacion'
]

In [ ]:
columnas_numericas_fests = [
    "Edicio",
    "Assistents_a_Barcelona",
    "Assistents_fora_de_Barcelona",
    "total_asist"
]